In [1]:
!pip install wandb -qU
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.4/881.4 kB 24.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Log in to your W&B account
import random
import wandb
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:


# Определяем общее количество экспериментов
total_runs = 5

# Запускаем несколько экспериментов
for run in range(total_runs):
    # 1️. Начинаем новый эксперимент и задаем его параметры
    wandb.init(
        project="yolo-segmentation-project",  # Название вашего проекта в W&B
        name=f"experiment_{run}",  # Название эксперимента
        config={
            "epochs": 100,  # Количество эпох
            "learning_rate": random.uniform(0.01, 0.03),  # Случайный learning rate для каждого эксперимента
            "batch_size": 16,  # Пример размера батча
        }
    )

    # Загружаем модель YOLO
    model = YOLO("/content/drive/MyDrive/merged_best.pt")

    # Функция для заморозки верхних слоев (если требуется)
    def freeze_layer(trainer):
        model = trainer.model
        num_freeze = 5  # Замораживаем 5 верхних слоёв
        print(f"Freezing {num_freeze} layers")
        freeze = [f'model.{x}.' for x in range(num_freeze)]
        for k, v in model.named_parameters():
            v.requires_grad = True
            if any(x in k for x in freeze):
                print(f'freezing {k}')
                v.requires_grad = False
        print(f"{num_freeze} layers are freezed.")

    # Добавляем callback для заморозки слоёв
    model.add_callback("on_train_start", freeze_layer)

    # 2️. Запускаем обучение модели с логированием в wandb
    model.train(
        data="/content/drive/MyDrive/all/finish_fish/data.yaml",
        epochs=wandb.config.epochs,  # Используем количество эпох из wandb.config
        project="yolo-segmentation-project",  # Название проекта
        name=f"experiment_{run}",  # Название эксперимента
        resume=False,
        batch=wandb.config.batch_size  # Используем размер батча из wandb.config
    )

    # Заканчиваем текущий эксперимент
    wandb.finish()


Ultralytics 8.3.2 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=/content/drive/MyDrive/merged_best.pt, data=/content/drive/MyDrive/all/finish_fish/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=864, save=True, save_period=-1, cache=False, device=None, workers=8, project=yolo-segmentation-project, name=experiment_06, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False,

train: Scanning /content/drive/MyDrive/all/finish_fish/data/labels/train.cache... 213 images, 0 backgrounds, 0 corrupt: 100%|██████████| 213/213 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/all/finish_fish/data/labels/val... 54 images, 0 backgrounds, 0 corrupt: 100%|██████████| 54/54 [02:22<00:00,  2.65s/it]

val: New cache created: /content/drive/MyDrive/all/finish_fish/data/labels/val.cache


...

     46/100      10.7G     0.4609     0.6844     0.4481     0.8822         57        864: 100%|██████████| 14/14 [00:16<00:00,  1.19s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.75s/it]

                   all         54        427      0.926      0.927      0.944      0.805      0.923      0.925       0.94      0.783



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


     47/100      10.4G     0.4569       0.68     0.4256     0.8822        129        864: 100%|██████████| 14/14 [00:16<00:00,  1.17s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.98s/it]

                   all         54        427      0.972      0.898      0.945      0.821      0.973      0.902      0.946      0.798
